In [11]:
import Contract
import Common
import Options
import Observable
import UnitsDB
import Calendar
import Swaps
import Credit
import Observations
import Valuation
import Interpreter
import Prelude hiding (and)
{-# LANGUAGE GADTs #-}
{-# LANGUAGE OverloadedStrings, ExtendedDefaultRules, ScopedTypeVariables #-}

In [3]:
underlying sp = (financial quantity usd cash) `and` (financial quantity eur cash)
              `and` give (financial (sp * 2 * quantity) gbp cash)
quantity = 100
usd = Currency "USD"
eur = Currency "EUR"
gbp = Currency "GBP"
cash = CashFlowType "cash"
exerciseDetails = europeanExercise (date 2018 06 01) strikePrice
strikePrice = 0.78

In [4]:
contract =
    option "opt" exerciseDetails CallOption emptyOptionAttrs underlying

In [5]:
contract

When (time == 2018-06-01 00:00:00 UTC) (Or "opt" (And (And (Scale 100.0 (One (Financial (Currency "USD") (CashFlowType "cash") Nothing))) (Scale 100.0 (One (Financial (Currency "EUR") (CashFlowType "cash") Nothing)))) (Give (Scale ((0.78 * 2.0) * 100.0) (One (Financial (Currency "GBP") (CashFlowType "cash") Nothing))))) Zero)

In [3]:
underlying = zcb (date 2021 06 01) 100000000 (Currency "USD") (CashFlowType "cash")

In [103]:
cparbEvent1 = NamedCond "cparbEvent1"
cparbEvent2 = NamedCond "cparbEvent2"
cpardfRate = NamedVal "cpardfRate"

In [104]:
cEvents = [("Bankruptcy", True, 'P', 50000000, Currency "USD", cparbEvent1),
                   ("Failure to Pay", False, 'C', 1000000, Currency "USD", cparbEvent2) ]

In [105]:
contract2 = creditDefaultSwap cEvents 1000000 
    [date 2019 01 01, date 2020 01 01, date 2021 01 01] (Currency "USD") cpardfRate (CashFlowType "cash") (90/360) underlying

In [21]:
:k Market

Market :: *

In [3]:
isTrue (date 2018 10 16) (at (date 2018 10 14))

False

In [8]:
take 10 $ unPr $ evalC (simpleModel (date 2017 10 14)) (Currency "USD") 
    (When (at $ date 2018 06 01) (Scale 100 (One (Financial (Currency "USD") (CashFlowType "cash") Nothing))))

[[100.0]]

In [13]:
:i disc

In [7]:
tempContract =
    option "opt" exerciseDetails CallOption emptyOptionAttrs underlying

  where
    -- note that the underlying contract describes *both* what we recieve and
    -- how much we pay for it (which depends on the strike price).
    -- of course for a vanilla european option the strike price is fixed,
    -- but with something like an asian option, it's calculated,
    -- which is why here the strike price is a paramater to underlying rather
    -- than us just using the 'strikePrice' variable defined below.

    underlying sp = physical quantity (mkt gas thm nbp)
              `and` give (financial (sp * quantity) gbp cash)
    quantity = 10

    -- Note also, that this 'sp' (strike price) paramater can be used as the
    -- basis for multiple cash flows, e.g. regular payments, rather than just
    -- a payment at the time the option is exercised.


    -- here we have a barrier condition on a different index to the price of
    -- the underlying, the barrier is based on the temperature where as the
    -- the underlying is for gas.
    temperatureUK = NamedVal "temperatureUK"
    gas = Commodity "gas"
    thm = Unit "thm"
    nbp = Location "nbp"
    gbp = Currency "gbp"
    cash = CashFlowType "cash"
    exerciseDetails =
      barrierDownAndIn temperatureUK floorTemp $
        europeanExercise (date 2018 06 01) strikePrice

    strikePrice = 2.5
    floorTemp   = 5 --degrees

In [8]:
tempContract

When (temperatureUK <= 5.0) (When (time == 2018-06-01 00:00:00 UTC) (Or "opt" (And (Scale 10.0 (One (Physical (Commodity "gas") (Unit "thm") (Location "nbp") Nothing Nothing))) (Give (Scale (2.5 * 10.0) (One (Financial (Currency "gbp") (CashFlowType "cash") Nothing))))) Zero))

In [12]:
:i evalObs